## Importing relevant libraries

In [5]:
import numpy as np
import pandas as pd

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer

## Reading of dataset

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/twitter-data/master/twitt30k.csv')
df

,twitts,sentiment
0,@robbiebronniman Sounds like a great night.,1
1,Damn the person who stolde my wallet !!!!! Ma...,1
2,Greetings from the piano bench (photo) http:/...,1
3,@drewryanscott i love it!! i love you!! haha f...,1
4,"@kissthestars Pretty pretty pretty please, pak...",0
...,...,...
29995,@Calumfan1 is it in any way related to photosh...,0
29996,@Swiz_NZ really? wow thats crap,0
29997,"At the 2010 lexus HS250h press event. Again, ...",0
29998,@karmicunderpath ooooh now there's a nice thou...,1


In [8]:
df['sentiment'].value_counts()

0    15000
1    15000
Name: sentiment, dtype: int64

## Training the model using SVM

In [9]:
def run_svm(df):
    X = df['twitts']
    y = df['sentiment']

    tfidf = TfidfVectorizer()

    X = tfidf.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

    print('Shape of X:', X.shape)

    svm_clf = LinearSVC()
    svm_clf.fit(X_train, y_train)

    y_pred = svm_clf.predict(X_test)

    print('Printing report')
    print(classification_report(y_test,y_pred))
    
    return tfidf, svm_clf

In [10]:
%%time
tfidf, clf= run_svm(df)

Shape of X: (30000, 40854)
Printing report
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000

CPU times: user 1.04 s, sys: 35.1 ms, total: 1.07 s
Wall time: 1.37 s


In [11]:
x = 'You are a good person. I hate you.'

clf.predict(tfidf.transform([x]))

array([1])

## Preprocessing using self built preprocessing package

In [12]:
import preprocess_kgptalkie as pp

In [13]:
pp.__version__

'0.10.3'

In [14]:
help(pp)

Help on package preprocess_kgptalkie:

NAME
    preprocess_kgptalkie

PACKAGE CONTENTS
    utils

FUNCTIONS
    cont_exp(x)
    
    get_avg_wordlength(x)
    
    get_basic_features(df)
    
    get_charcounts(x)
    
    get_digit_counts(x)
    
    get_emails(x)
    
    get_hashtag_counts(x)
    
    get_mentions_counts(x)
    
    get_ngram(df, col, ngram_range)
    
    get_stopwords_counts(x)
    
    get_uppercase_counts(x)
    
    get_urls()
    
    get_value_counts(df, col)
    
    get_word_freqs(df, col)
    
    get_wordcounts(x)
    
    make_base(x)
    
    remove_accented_chars(x)
    
    remove_common_words(x, freq, n=20)
    
    remove_dups_char(x)
    
    remove_emails(x)
    
    remove_html_tags(x)
    
    remove_rarewords(x, freq, n=20)
    
    remove_rt(x)
    
    remove_special_chars(x)
    
    remove_stopwords(x)
    
    remove_urls(x)
    
    spelling_correction(x)

VERSION
    0.10.3

FILE
    /Users/anchitkhanna/opt/anaconda3/envs/py3-TF2.0/lib/p

In [15]:
df['twitts'] = df['twitts'].apply(lambda x: x.lower())

In [16]:
df['twitts'] = df['twitts'].apply(lambda x: pp.cont_exp(x))

In [17]:
df['twitts'].sample(5)

26640                         i have so much things to do 
14765                 i hope i can talk to my boo tonight 
28190                        what are you thinking about? 
27966               @rixgerald hey man f you! i hate this 
12909    getting ready to go pick up my brand new lappy...
Name: twitts, dtype: object

In [18]:
df.head()

,twitts,sentiment
0,@robbiebronniman sounds like a great night.,1
1,damn the person who stolde my wallet !!!!! ma...,1
2,greetings from the piano bench (photo) http:/...,1
3,@drewryanscott i love it!! i love you!! haha f...,1
4,"@kissthestars pretty pretty pretty please, pak...",0


In [19]:
tfidf, clf = run_svm(df)

Shape of X: (30000, 40753)
Printing report
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.75      0.76      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000



In [20]:
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_emails(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_urls(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_rt(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_html_tags(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_special_chars(x))

In [21]:
tfidf, clf = run_svm(df)

Shape of X: (30000, 42855)
Printing report
              precision    recall  f1-score   support

           0       0.75      0.74      0.74      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000



In [22]:
x = 'You are a good person. I hate you.'

clf.predict(tfidf.transform([x]))

array([1])

## Fine tuning SVM model

In [23]:
def run_svm(df):
    X = df['twitts']
    y = df['sentiment']

    tfidf = TfidfVectorizer(norm = 'l1', ngram_range=(1,2), analyzer='word', max_features=5000)

    X = tfidf.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

    print('Shape of X:', X.shape)

    svm_clf = LinearSVC()
    svm_clf.fit(X_train, y_train)

    y_pred = svm_clf.predict(X_test)

    print('Printing report')
    print(classification_report(y_test,y_pred))
    
    return tfidf, svm_clf

run_svm(df)

Shape of X: (30000, 5000)
Printing report
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      3000
           1       0.76      0.74      0.75      3000

    accuracy                           0.76      6000
   macro avg       0.76      0.76      0.76      6000
weighted avg       0.76      0.76      0.76      6000



(TfidfVectorizer(max_features=5000, ngram_range=(1, 2), norm='l1'),
 LinearSVC())

## Saving and Loading the model using pickle

In [24]:
import pickle

In [25]:
pickle.dump(clf, open('clf.pkl','wb'))
pickle.dump(tfidf, open('tfidf.pkl','wb'))

In [39]:
del clf
del tfidf

In [27]:
clf = pickle.load(open('clf.pkl','rb'))
tfidf = pickle.load(open('tfidf.pkl','rb'))

In [28]:
clf

LinearSVC()

In [29]:
# tfidf.vocabulary_   it is to check the tokenization of words. It shows how each word is assigned a specific token. 

In [30]:
clf.predict(tfidf.transform([x]))

array([1])

## Real Time Twitter Sentiment Analysis

In [31]:
consumer_key = 'JK7J6K9JLqpjVSWECQxFRSiMf'
consumer_secret = 'SaDQHh6ihLR67w3GeXTcgIV2fUHBd5TOGS9MPa6UiXpKkVuJtZ'
access_token = '1360377026071457802-jTRegF0GkMWf13vv2guJjjsBPtQVNy'
access_token_secret = 'cGxn2ho7DddrTIotjVO9S86kJwjlBJzRv4CNwjnWj92Kr'

In [32]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()

In [33]:
for tweet in public_tweets:
    print(tweet.text)

Paper Explained: NFNets - High-Performance Large-Scale Image Recognition Without Normalization (Full Video Analysis… https://t.co/wXT0gWkGVl
RT @IrishTimesSport: Solskjaer determined Man United will rekindle title challenge https://t.co/eJ7ucrCwim via @IrishTimesSport
‘We want closure’: Stardust victims’ families mark 40th anniversary https://t.co/fTwmThCuam
Lionel Messi's team-mates’ reaction when Barcelona superstar told him he is 'hated'
https://t.co/xFPLmPV8we
RT @IrishTimesSport: Irish raiders set to dominate Cheltenham again https://t.co/eracNZ1Bdv via @IrishTimesSport
Competition: enter now for your chance to win a love inspired hamper including a bottle of Grace O’Malley award win… https://t.co/vi8AIsWk5C
Patrick Bamford reiterates desire to play for England rather than Ireland https://t.co/sXNlwvD9rF
RT @IrishTimesWorld: Republicans need to convince Trump he cannot win in 2024 https://t.co/CHfnan5ETF via @IrishTimesWorld
RT @irishtimesvideo: ► VIDEO: ‘There’s a huge amount of 

## Tracking Kewords

In [55]:
import json
from textblob import TextBlob
import csv

In [40]:
clf = pickle.load(open('clf.pkl','rb'))
tfidf = pickle.load(open('tfidf.pkl','rb'))

In [41]:
def predict_sentiment(x):
    x = [x]
    sent = clf.predict(tfidf.transform(x))
    return sent 

In [42]:
predict_sentiment('what i am not best')[0]

0

In [47]:
keyword = ['usa', 'china']

In [62]:
usa = 0
china = 0

with open('sentiment.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=keyword)
    writer.writeheader()
    
class MyStreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        print(status.text)
    
    def on_data(self, data):
        raw_tweets = json.loads(data)
        #print(raw_tweets)
        try:
            x = str(raw_tweets['text']).lower()
            x = pp.cont_exp(x)
            x = pp.remove_emails(x)
            x = pp.remove_urls(x)
            x = pp.remove_html_tags(x)
            x = pp.remove_rt(x)
            x = pp.remove_special_chars(x)
            
            #blob = TextBlob(x)
#             sentiment = predict_sentiment(x)[0]
#             print(sentiment)
            
            global usa
            global china
            
            if 'usa' in x and 'china' not in x:
                sent = predict_sentiment(x)[0]
                usa = usa + sent
            elif 'china' in x and 'usa' not in x:
                sent = predict_sentiment(x)[0]
                china = china + sent
            else:
                pass
            
            print('usa: ',usa, 'china: ',china)
            
            with open('sentiment.csv', 'a') as file:
                writer = csv.DictWriter(file, fieldnames=keyword)
                info = {
                    'usa': usa,
                    'china': china
                }
                writer.writerow(info)
            
        except:
            pass
    
    def on_error(self, status_code):
        if status_code == 420:
            print('Error 420')
            #returning False in on_error disconnects the stream
            return False

In [63]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [64]:
myStream.filter(track=keyword)

usa:  0 china:  0
usa:  0 china:  1
usa:  0 china:  1
usa:  0 china:  2
usa:  0 china:  2
usa:  0 china:  2
usa:  0 china:  3
usa:  0 china:  3
usa:  0 china:  3
usa:  1 china:  3
usa:  2 china:  3
usa:  3 china:  3
usa:  4 china:  3
usa:  5 china:  3
usa:  5 china:  3
usa:  6 china:  3
usa:  6 china:  3
usa:  6 china:  3
usa:  7 china:  3
usa:  7 china:  3
usa:  7 china:  4
usa:  7 china:  4
usa:  7 china:  4
usa:  7 china:  4
usa:  7 china:  4
usa:  8 china:  4
usa:  8 china:  5
usa:  8 china:  6
usa:  8 china:  6
usa:  8 china:  7
usa:  9 china:  7
usa:  9 china:  8
usa:  9 china:  8
usa:  9 china:  8
usa:  9 china:  8
usa:  10 china:  8
usa:  10 china:  8
usa:  10 china:  8
usa:  11 china:  8
usa:  11 china:  8
usa:  11 china:  8
usa:  12 china:  8
usa:  12 china:  8
usa:  12 china:  9
usa:  12 china:  9
usa:  12 china:  9
usa:  12 china:  9
usa:  12 china:  9
usa:  12 china:  9
usa:  12 china:  9
usa:  12 china:  9
usa:  12 china:  9
usa:  12 china:  9
usa:  13 china:  9
usa:  14 

usa:  108 china:  69
usa:  108 china:  70
usa:  108 china:  70
usa:  108 china:  71
usa:  109 china:  71
usa:  109 china:  72
usa:  109 china:  73
usa:  109 china:  73
usa:  110 china:  73
usa:  111 china:  73
usa:  111 china:  73
usa:  111 china:  73
usa:  111 china:  73
usa:  111 china:  73
usa:  111 china:  73
usa:  112 china:  73
usa:  113 china:  73
usa:  114 china:  73
usa:  114 china:  73
usa:  114 china:  74
usa:  115 china:  74
usa:  115 china:  74
usa:  115 china:  75
usa:  115 china:  75
usa:  115 china:  75
usa:  116 china:  75
usa:  116 china:  76
usa:  116 china:  76
usa:  116 china:  76
usa:  116 china:  76
usa:  116 china:  76
usa:  116 china:  77
usa:  116 china:  77
usa:  116 china:  77
usa:  116 china:  77
usa:  116 china:  77
usa:  116 china:  78
usa:  116 china:  78
usa:  116 china:  78
usa:  116 china:  78
usa:  116 china:  78
usa:  116 china:  79
usa:  116 china:  79
usa:  116 china:  79
usa:  116 china:  79
usa:  116 china:  79
usa:  116 china:  79
usa:  116 chi

usa:  201 china:  136
usa:  201 china:  137
usa:  201 china:  137
usa:  201 china:  138
usa:  202 china:  138
usa:  202 china:  138
usa:  202 china:  138
usa:  202 china:  138
usa:  202 china:  138
usa:  202 china:  138
usa:  202 china:  139
usa:  203 china:  139
usa:  203 china:  140
usa:  204 china:  140
usa:  204 china:  141
usa:  204 china:  141
usa:  204 china:  142
usa:  204 china:  142
usa:  205 china:  142
usa:  205 china:  142
usa:  206 china:  142
usa:  206 china:  142
usa:  207 china:  142
usa:  207 china:  142
usa:  207 china:  143
usa:  207 china:  144
usa:  207 china:  144
usa:  207 china:  144
usa:  207 china:  144
usa:  207 china:  145
usa:  207 china:  146
usa:  207 china:  147
usa:  208 china:  147
usa:  208 china:  148
usa:  208 china:  148
usa:  208 china:  148
usa:  209 china:  148
usa:  210 china:  148
usa:  210 china:  149
usa:  211 china:  149
usa:  211 china:  149
usa:  211 china:  149
usa:  211 china:  149
usa:  211 china:  149
usa:  212 china:  149
usa:  212 

usa:  288 china:  213
usa:  288 china:  214
usa:  288 china:  214
usa:  288 china:  214
usa:  288 china:  215
usa:  289 china:  215
usa:  289 china:  215
usa:  289 china:  215
usa:  289 china:  215
usa:  289 china:  215
usa:  289 china:  216
usa:  289 china:  216
usa:  289 china:  216
usa:  289 china:  217
usa:  289 china:  218
usa:  289 china:  218
usa:  290 china:  218
usa:  290 china:  218
usa:  290 china:  218
usa:  291 china:  218
usa:  291 china:  219
usa:  291 china:  219
usa:  292 china:  219
usa:  292 china:  219
usa:  292 china:  219
usa:  292 china:  219
usa:  293 china:  219
usa:  294 china:  219
usa:  294 china:  219
usa:  294 china:  220
usa:  294 china:  220
usa:  294 china:  220
usa:  294 china:  220
usa:  294 china:  221
usa:  294 china:  221
usa:  295 china:  221
usa:  296 china:  221
usa:  296 china:  222
usa:  296 china:  222
usa:  296 china:  222
usa:  296 china:  222
usa:  297 china:  222
usa:  297 china:  222
usa:  297 china:  222
usa:  297 china:  222
usa:  297 

usa:  390 china:  287
usa:  391 china:  287
usa:  391 china:  287
usa:  391 china:  287
usa:  392 china:  287
usa:  393 china:  287
usa:  393 china:  287
usa:  393 china:  287
usa:  393 china:  287
usa:  393 china:  288
usa:  394 china:  288
usa:  394 china:  288
usa:  394 china:  288
usa:  394 china:  288
usa:  394 china:  289
usa:  394 china:  290
usa:  394 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  395 china:  290
usa:  396 china:  290
usa:  396 china:  291
usa:  396 china:  291
usa:  396 china:  291
usa:  397 china:  291
usa:  398 china:  291
usa:  398 china:  291
usa:  398 china:  291
usa:  398 china:  291
usa:  398 china:  291
usa:  398 china:  291
usa:  398 china:  291
usa:  398 china:  291
usa:  398 china:  291
usa:  398 china:  291
usa:  398 china:  292
usa:  399 china:  292
usa:  399 

...

usa:  726 china:  627
usa:  726 china:  628
usa:  726 china:  628
usa:  726 china:  628
usa:  726 china:  628
usa:  726 china:  628
usa:  726 china:  628
usa:  726 china:  628
usa:  727 china:  628
usa:  727 china:  628
usa:  727 china:  628
usa:  727 china:  628
usa:  727 china:  628
usa:  728 china:  628
usa:  728 china:  628
usa:  728 china:  629
usa:  729 china:  629
usa:  729 china:  629
usa:  729 china:  629
usa:  730 china:  629
usa:  730 china:  629
usa:  730 china:  630
usa:  730 china:  630
usa:  730 china:  630
usa:  730 china:  631
usa:  730 china:  631
usa:  731 china:  631
usa:  731 china:  631
usa:  731 china:  631
usa:  731 china:  631
usa:  731 china:  631
usa:  732 china:  631
usa:  732 china:  631
usa:  732 china:  632
usa:  732 china:  633
usa:  732 china:  633
usa:  733 china:  633
usa:  733 china:  634
usa:  733 china:  634
usa:  733 china:  634
usa:  733 china:  634
usa:  734 china:  634
usa:  734 china:  634
usa:  735 china:  634
usa:  736 china:  634
usa:  736 

usa:  816 china:  691
usa:  817 china:  691
usa:  817 china:  691
usa:  817 china:  691
usa:  817 china:  691
usa:  817 china:  691
usa:  818 china:  691
usa:  818 china:  691
usa:  819 china:  691
usa:  819 china:  691
usa:  820 china:  691
usa:  821 china:  691
usa:  821 china:  692
usa:  821 china:  692
usa:  821 china:  693
usa:  822 china:  693
usa:  823 china:  693
usa:  823 china:  693
usa:  823 china:  694
usa:  824 china:  694
usa:  824 china:  694
usa:  824 china:  694
usa:  824 china:  694
usa:  824 china:  694
usa:  824 china:  694
usa:  824 china:  695
usa:  825 china:  695
usa:  826 china:  695
usa:  827 china:  695
usa:  828 china:  695
usa:  828 china:  695
usa:  828 china:  695
usa:  828 china:  695
usa:  828 china:  695
usa:  828 china:  696
usa:  828 china:  696
usa:  828 china:  696
usa:  828 china:  697
usa:  828 china:  697
usa:  828 china:  697
usa:  829 china:  697
usa:  830 china:  697
usa:  831 china:  697
usa:  831 china:  697
usa:  831 china:  697
usa:  831 

usa:  905 china:  770
usa:  905 china:  771
usa:  905 china:  772
usa:  906 china:  772
usa:  906 china:  772
usa:  906 china:  772
usa:  906 china:  772
usa:  906 china:  772
usa:  906 china:  772
usa:  906 china:  772
usa:  907 china:  772
usa:  908 china:  772
usa:  908 china:  773
usa:  908 china:  774
usa:  909 china:  774
usa:  909 china:  774
usa:  909 china:  774
usa:  909 china:  775
usa:  910 china:  775
usa:  910 china:  776
usa:  911 china:  776
usa:  912 china:  776
usa:  912 china:  776
usa:  913 china:  776
usa:  913 china:  776
usa:  913 china:  777
usa:  913 china:  777
usa:  913 china:  777
usa:  913 china:  778
usa:  914 china:  778
usa:  914 china:  778
usa:  914 china:  778
usa:  914 china:  778
usa:  915 china:  778
usa:  915 china:  779
usa:  915 china:  779
usa:  915 china:  779
usa:  916 china:  779
usa:  917 china:  779
usa:  917 china:  779
usa:  917 china:  779
usa:  917 china:  779
usa:  918 china:  779
usa:  918 china:  779
usa:  918 china:  779
usa:  918 

usa:  1001 china:  853
usa:  1002 china:  853
usa:  1002 china:  853
usa:  1002 china:  853
usa:  1002 china:  853
usa:  1003 china:  853
usa:  1003 china:  853
usa:  1003 china:  853
usa:  1004 china:  853
usa:  1004 china:  854
usa:  1004 china:  854
usa:  1004 china:  854
usa:  1005 china:  854
usa:  1005 china:  855
usa:  1005 china:  855
usa:  1005 china:  855
usa:  1005 china:  855
usa:  1006 china:  855
usa:  1007 china:  855
usa:  1007 china:  856
usa:  1008 china:  856
usa:  1009 china:  856
usa:  1009 china:  856
usa:  1009 china:  856
usa:  1009 china:  856
usa:  1009 china:  856
usa:  1010 china:  856
usa:  1010 china:  857
usa:  1010 china:  857
usa:  1010 china:  857
usa:  1010 china:  858
usa:  1010 china:  858
usa:  1010 china:  858
usa:  1011 china:  858
usa:  1011 china:  858
usa:  1012 china:  858
usa:  1012 china:  858
usa:  1012 china:  859
usa:  1012 china:  859
usa:  1013 china:  859
usa:  1014 china:  859
usa:  1014 china:  859
usa:  1015 china:  859
usa:  1016 

KeyboardInterrupt: 